In [14]:
import pandas as pd
import numpy as np

from langdetect import detect
import tqdm

In [43]:
data_path = "../data/once_dialogues/scored_data.csv"

data = pd.read_csv(data_path)
cols = list(data.columns)[1:]
data = data[cols]
data = data[data["text"].apply(lambda x: len(x) > 5)]
data.head()

,text,Flirt,Social Network,Request/Invite to Meet/Date,Request for Phone Number,Normal Response/Question/Sentence,session_id
0,"hah, we need our rest so we can go wild",0.356138,0.184912,0.181820,0.143495,0.133634,94075319153
1,"that would be fun, I like naps baby, u know it",0.314385,0.083779,0.227397,0.077000,0.297439,94075319153
2,"oh baby I would slide my cock deep inside u, j...",0.436363,0.114463,0.198143,0.076235,0.174796,94075319153
3,oh that would make it even easier haha,0.278040,0.156456,0.235906,0.172829,0.156770,94075319153
4,"damn baby, does that make u feel sexy",0.565976,0.087618,0.154389,0.084188,0.107829,94075319153


In [44]:
cols

['text',
 'Flirt',
 'Social Network',
 'Request/Invite to Meet/Date',
 'Request for Phone Number',
 'Normal Response/Question/Sentence',
 'session_id']

In [45]:
one_col_data = {}
stats = {}

col_names = [
    "Flirt",
    "Social Network",
    "Request/Invite to Meet/Date",
    "Request for Phone Number",
    "Normal Response/Question/Sentence",
]

params = {
    "Flirt": {"thersholds": 0.7, "to_sample": 2000},
    "Social Network": {"thersholds": 0.4, "to_sample": 100},
    "Request/Invite to Meet/Date": {"thersholds": 0.6, "to_sample": 800},
    "Request for Phone Number": {"thersholds": 0.5, "to_sample": 450},
    "Normal Response/Question/Sentence": {"thersholds": 0.7, "to_sample": 2000},
}

for name in col_names:
    one_col_data[name] = data[["text", name, "session_id"]].sort_values(name, ascending=False)
    one_col_data[name] = one_col_data[name][one_col_data[name][name] > params[name]["thersholds"]]
    t = [detect(l) for l in tqdm.tqdm(one_col_data[name]["text"].tolist())]
    one_col_data[name]["lang"] = t
    one_col_data[name] = one_col_data[name][one_col_data[name]["lang"] == "en"]
    one_col_data[name] = one_col_data[name].sample(n=params[name]["to_sample"], random_state=42)
    print(name, one_col_data[name].shape)

100%|██████████| 3264/3264 [00:15<00:00, 215.67it/s]


Flirt (2000, 4)


100%|██████████| 162/162 [00:00<00:00, 224.81it/s]


Social Network (100, 4)


 27%|██▋       | 424/1579 [00:01<00:03, 299.72it/s]

...

Request/Invite to Meet/Date (800, 4)


100%|██████████| 584/584 [00:02<00:00, 213.17it/s]


Request for Phone Number (450, 4)


100%|██████████| 16268/16268 [00:57<00:00, 282.23it/s]

Normal Response/Question/Sentence (2000, 4)


In [46]:
for key in one_col_data:
    one_col_data[key] = one_col_data[key].sort_values(key, ascending=False)
    one_col_data[key]["class_name"] = key
    c = ["text", "score", "session_id", "lang", "class_name"]
    one_col_data[key].columns = c
    one_col_data[key] = one_col_data[key][["session_id", "text", "lang", "class_name", "score"]]

In [50]:
new_data = pd.concat([one_col_data[t] for t in one_col_data], ignore_index=True)
new_data.to_csv("../data/once_dialogues/scored_selected_data.csv")